# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api

db_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


cuda is available.


In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
param1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list":['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list":["value"]
    },
    'data_y_flag' : 'true',
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# forecasting
param2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'false',
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

# classification
param3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list":['Temperature', 'out_temp','sin_hour']
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list":['Temperature']
    },
    'data_y_flag' : 'true',
    'model_name':'regression_energy_cleanLevel4_trainX_regression_LSTM_rg_'
}

In [3]:
params = param1

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [4]:
# get model meta
model_name = params['model_name']
model_meta = mongodb_client.get_document_by_json('model', 'meta',{'model_info.model_name':model_name} )
model_meta = model_meta[0]

In [5]:
model_meta

{'ingestion_param_X': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainX',
  'feature_list': ['Press_mm_hg',
   'RH_1',
   'RH_2',
   'RH_3',
   'RH_4',
   'RH_5',
   'RH_6',
   'RH_7',
   'RH_8',
   'RH_9',
   'RH_out',
   'T1',
   'T2',
   'T3',
   'T4',
   'T5',
   'T6',
   'T7',
   'T8',
   'T9',
   'T_out',
   'Tdewpoint',
   'Visibility',
   'Windspeed']},
 'ingestion_param_y': {'bucket_name': 'integration',
  'ms_name': 'regression_energy_cleanLevel4_trainy',
  'feature_list': ['value']},
 'data_y_flag': True,
 'scaler_param': {'scaler_flag': 'scale',
  'scale_method': 'minmax',
  'scaler_path': './scaler/',
  'scaler_file_path': {'XScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainX/minmax/900878d7bba6f2a4017d1c3399909ea7/scaler.pkl'},
   'yScalerFile': {'fileName': 'scaler.pkl',
    'filePath': './scaler/regression_energy_cleanLevel4_trainy/minmax/f69156750a210491ffd4a67b605bc88b/scaler.pkl'}}},


In [6]:
# make test data, scaler
test_X_array, test_y_array, scaler_X, scaler_y = ML_api.test_data_preparation(params, model_meta, db_client)

KeyError: 'data_y_flag'

In [ ]:
# set scaler, feature list by data y flag
scaler_feature_dict = {}
if model_meta['data_y_flag']:
    scaler_feature_dict['scaler'] = scaler_y
    scaler_feature_dict['feature_list'] = model_meta['ingestion_param_y']['feature_list']
    scaler_feature_dict['target'] = model_meta['ingestion_param_y']['feature_list'][0]

else:
    scaler_feature_dict['scaler'] = scaler_X
    scaler_feature_dict['feature_list'] = model_meta['ingestion_param_X']['feature_list']
    scaler_feature_dict['target'] = model_meta['ingestion_param_X']['feature_list'][0]

In [ ]:
# model test
result = ML_api.ML_test(model_meta, test_X_array, test_y_array, scaler_feature_dict)

In [ ]:
result

In [ ]:
model_meta['model_info']